# CO2 Especification Detector

- vamos a crear un modelo de IA para determinar el nivel de pureza de CO2 en el ambito alimenticio, dado por un analisis -ya sea de forma manual o por pdf.- y dar un aviso si pasa de las especificaciones del cliente.

- Datos a tener en cuenta:
    - Para esta primera versión voy a enfocarme en el CO2 liquido.

- Pureza de CO2:
    - 99,9 %

- Niveles de Impurezas permitidos:
    - Humedad (H2O) <= 50 ppm v/v
    - Oxigeno (O2) <= 30 ppm v/v
    - Monoxido de Carbono (CO) <= 10 ppm v/v
    - Óxidos de Nitrogeno (NOx) <= 10 ppm v/v
    - Compuestos Organicos no Volatiles (COV) <= 5ppm v/v
    - Hidrocarburos (CnHm) <= 30 ppm v/v
    - Azufre Total (SO2) <= 0.05 ppm v/v

--------

- NOTA:
    - ppm v/v : Partes por millon en volumen.
    - ppm w/w : Partes por millon en peso.

- Link publico de AirLiquide con las especificaciones que utilizan:
    - https://es.airliquide.com/gases/gases-de-calidad-alimentaria-aligal/aligaltm-2-dioxido-de-carbono-co2-de-calidad-alimentaria

- Fecha del Notebook:
    - 26/02/25

# Creación de Datos Sinteticos:

- Al no encontrar datos publicos para esta primera versión vamos a generar datos sinteticos para entrenar al modelo. Es importante tenerlo en cuenta aunque si se entrenan correctamente con las especificaciones correctas debería de dar un modelo funcional.

In [145]:

# Librerías:

import numpy as np
import pandas as pd

In [146]:
# Generador de Datos Sinteticos

def DataGeneration(max_value, crit_value, seed, min_value= 0, p_outranged = 0.2, n = 10000):

    #Para poder replicar los datos, vamos a establecer una semilla.
    random_seed = np.random.seed(seed)
    
    #Determinamos los datos que estaran o no dentro de las especifiaciones.
    ranged = int (n * (1- p_outranged))
    outranged = int (n - ranged)

    #Generamos los datos. 
    data_ranged = np.random.uniform(min_value, max_value, ranged)
    data_outranged = np.random.uniform(min_value, crit_value, outranged)

    #Juntamos y mezclamos
    data = np.concatenate([data_ranged, data_outranged])
    np.random.shuffle(data)

    return data

- Explicacion de los args:

    - min_value: Valor minimo permitido en las especificaciones. Por defecto 0.
    - max_value: Valor máximo permitido en las especificaciones. Esto hay que ponerlo según el producto.
    - crit_value: Valor superior al permitido en las especificaciones para crear un dataset con datos erroreos, es importante para que el modelo pueda aprender.
    - p_outranged: % de datos fuera del rango. Por defecto el 20%
    - n: numero de muestras, por defecto 1000.

In [147]:
#Generamos los datos

co2_purify = DataGeneration(min_value = 99.9, max_value = 100, crit_value = 99.8, seed=42)
h2o = DataGeneration(max_value = 50, crit_value = 60, seed=42)
o2 = DataGeneration(max_value = 30, crit_value = 40, seed=42)
co = DataGeneration(max_value = 10, crit_value = 20, seed=42)
nox = DataGeneration(max_value = 10, crit_value = 20, seed=42)
cov = DataGeneration(max_value = 5, crit_value = 10, seed=42)
cnhm = DataGeneration(max_value = 30, crit_value = 40, seed=42)
so2 = DataGeneration(max_value = 0.05, crit_value = 0.1, seed=42)

data = pd.DataFrame({"co2purify" : co2_purify,
                     "h2o_ppm" : h2o,
                     "o2_ppm" : o2,
                     "co_ppm" : co,
                     "nox_ppm" : nox,
                     "cov_ppm": cov,
                     "cnhm_ppm" : cnhm,
                     "so2_ppm" : so2})

data.head()

,co2purify,h2o_ppm,o2_ppm,co_ppm,nox_ppm,cov_ppm,cnhm_ppm,so2_ppm
0,99.975338,37.668909,22.601346,7.533782,7.533782,3.766891,22.601346,0.037669
1,99.943235,21.617311,12.970387,4.323462,4.323462,2.161731,12.970387,0.021617
2,99.979781,39.890682,23.934409,7.978136,7.978136,3.989068,23.934409,0.039891
3,99.982825,41.412463,24.847478,8.282493,8.282493,4.141246,24.847478,0.041412
4,99.992069,46.034570,27.620742,9.206914,9.206914,4.603457,27.620742,0.046035


- Como este dataset es sintetico y la idea principal es que se inserten datos reales vamos a guardar el dataset en un csv y cargarlo para en un futuro poder empezar el proceso desde ese punto.

In [148]:
data.to_csv("../data/raw/sintetic_data.csv", sep=";", index=False)

# Carga de Datos:

In [149]:
def DataLoader():
    data = pd.read_csv("../data/raw/sintetic_data.csv", sep=";")
    return data

raw_data = DataLoader()

raw_data

,co2purify,h2o_ppm,o2_ppm,co_ppm,nox_ppm,cov_ppm,cnhm_ppm,so2_ppm
0,99.975338,37.668909,22.601346,7.533782,7.533782,3.766891,22.601346,0.037669
1,99.943235,21.617311,12.970387,4.323462,4.323462,2.161731,12.970387,0.021617
2,99.979781,39.890682,23.934409,7.978136,7.978136,3.989068,23.934409,0.039891
3,99.982825,41.412463,24.847478,8.282493,8.282493,4.141246,24.847478,0.041412
4,99.992069,46.034570,27.620742,9.206914,9.206914,4.603457,27.620742,0.046035
...,...,...,...,...,...,...,...,...
9995,99.993992,46.996219,28.197731,9.399244,9.399244,4.699622,28.197731,0.046996
9996,99.901469,0.734669,0.440802,0.146934,0.146934,0.073467,0.440802,0.000735
9997,99.901603,0.801408,0.480845,0.160282,0.160282,0.080141,0.480845,0.000801
9998,99.996854,48.426867,29.056120,9.685373,9.685373,4.842687,29.056120,0.048427


- Una vez cargado los datos, vamos a crear una nueva columna de validación donde se determinará si el Analisis entra dentro de las especificaciones o no.

In [155]:
def ValidationTest(dataset):
    #Pasamos un filtro con las especificaciones y lo convertimos 1 y 0
    #1 True
    #0 False
    dataset["validation"] = (
        (dataset["co2purify"] >= 99.9) &
        (dataset["h2o_ppm"] <= 50) &
        (dataset["o2_ppm"] <= 30) &
        (dataset["co_ppm"] <= 10) &
        (dataset["nox_ppm"] <= 10) &
        (dataset["cov_ppm"] <= 5) &
        (dataset["cnhm_ppm"] <= 30) &
        (dataset["so2_ppm"] <= 0.05)
    ).astype(int)

    return dataset

val_data = ValidationTest(raw_data)

val_data["validation"].value_counts()

validation
1    8000
0    2000
Name: count, dtype: int64